In [ ]:

from google.colab import drive
import os
import shutil
import glob
import yaml


drive.mount('/content/drive')


!pip install -q ultralytics

BASE_PATH = '/content/drive/MyDrive/inDrive_hackathon'
MODELS_PATH = os.path.join(BASE_PATH, 'models')
os.makedirs(MODELS_PATH, exist_ok=True)


RAW_ZIP_PATH = os.path.join(BASE_PATH, 'data', 'raw', 'safety_dataset_v2.zip')
DATASET_DIR = '/content/safety_dataset'
if os.path.exists(DATASET_DIR):
    shutil.rmtree(DATASET_DIR)
shutil.unpack_archive(RAW_ZIP_PATH, DATASET_DIR)
print(f"Dataset unzipped to {DATASET_DIR}")

DATA_YAML_PATH = os.path.join(DATASET_DIR, 'data.yaml')
if not os.path.exists(DATA_YAML_PATH):
    print("❌ ERROR: data.yaml file not found in unzipped folder!")
else:
    print("✅ Setup complete! data.yaml found.")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.4 MB/s eta 0:00:00
Dataset unzipped to /content/safety_dataset
✅ Setup complete! data.yaml found.


In [ ]:


with open(DATA_YAML_PATH, 'r') as f:
    data = yaml.safe_load(f)

data['path'] = DATASET_DIR
data['train'] = os.path.join(DATASET_DIR, 'train', 'images')
data['val'] = os.path.join(DATASET_DIR, 'valid', 'images')
data['test'] = os.path.join(DATASET_DIR, 'test', 'images')
with open(DATA_YAML_PATH, 'w') as f:
    yaml.dump(data, f, sort_keys=False)

print("\n--- Contents of updated data.yaml ---")
with open(DATA_YAML_PATH, 'r') as f:
    print(f.read())
print("\n✅ data.yaml paths updated for Colab environment.")


--- Contents of updated data.yaml ---
train: /content/safety_dataset/train/images
val: /content/safety_dataset/valid/images
test: /content/safety_dataset/test/images
nc: 3
names:
- cracked_windshield
- headlight_oxidation
- tire_damage
roboflow:
  workspace: indrivehackathon
  project: indrive-safety-qimgf
  version: 2
  license: CC BY 4.0
  url: https://universe.roboflow.com/indrivehackathon/indrive-safety-qimgf/dataset/2
path: /content/safety_dataset


✅ data.yaml paths updated for Colab environment.


In [ ]:

from ultralytics import YOLO


model = YOLO('yolov8n.pt')


results = model.train(
    data=DATA_YAML_PATH,
    epochs=75,
    imgsz=640,
    project=MODELS_PATH,
    name='safety_model_run1'
)

print("\n🎉 Training complete!")
print(f"Best safety model saved to: {MODELS_PATH}/safety_model_run1")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/safety_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ke